In [ ]:
import os
import pandas as pd
import numpy as np
import glob
import warnings
from __future__ import absolute_import, division, print_function, unicode_literals
import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
import pandas as pd
from download import download
import plotly.express as px
warnings.filterwarnings("ignore")

os.chdir(r"C:\\Users\\Tobi\\Desktop\\New folder\\git\\ETL_ABMF\\ML work")

globbed_files = glob.glob("*.csv")

# globbed_files

data = []
for csv in globbed_files:
    frame = pd.read_csv(csv)
    frame['filename'] = os.path.basename(csv)
    data.append(frame)
    df = pd.concat(data)
    
df.drop(columns=['filename'], axis=1, inplace=True)    

    

def plot(df,time:str(),col:str()):
    mpl.rcParams['figure.figsize']=(8,6)
    mpl.rcParams['axes.grid']=False

    fig = px.line(df, x = time, y=col,title= time + " vs " + col)

    fig.update_xaxes(
                    rangeslider_visible=True,
                    rangeselector = dict(
                                        buttons = list([
                                            dict(count=1, label="1y", step="year", stepmode = "backward"),
                                            dict(step='all')
                                        ]))
                    )
    return fig

def working_time():
    working_hrs = pd.date_range('2022-09-01 08:00:00','2022-09-01 17:00:00', freq='1min')
    working_hrs.freq=None
    working_hrs.astype(str)

    working_hrs_df = pd.DataFrame(working_hrs, columns=['Time'])
    working_hrs_df['new_date'] = [d.date() for d in working_hrs_df['Time']]
    working_hrs_df['new_time'] = [d.time() for d in working_hrs_df['Time']]

    working_hrs_df.drop(columns=['Time', 'new_date'], inplace=True)

    working_times = []
    for time in working_hrs_df.new_time:
        t = time.strftime("%H:%M:%S")
        working_times.append(t)
    return working_times

# working_time()

def reformed_df(df):
    
    df['Time']= pd.to_datetime(df['Time'])

    df['Time'] = df['Time'].dt.floor('T')

    df['Time'] = df['Time'].dt.floor('Min')

    df['new_date'] = [d.date() for d in df['Time']]
    df['new_time'] = [d.time() for d in df['Time']]
    df['new_time'] = df['new_time'].astype(str)
#     current_time_frame(df)
#     df['Time'].duplicated().any()
#     df['Time']= pd.to_datetime(df['Time'])
    df['Day_of_week']=df.Time.dt.strftime('%A')
#     df = df[~df.Day_of_week.isin(['Saturday','Sunday'])] 
#     df = df[df.new_time.isin(working_time())] 
    df = df.set_index('Time')
    df.index = pd.to_datetime(df.index)

    return(df)
# df.drop_duplicates()

df = reformed_df(df)
# df

def supposed_time_frame(df):
    time_frame = pd.date_range(df.index.min(),df.index.max(), freq='1min')
    time_frame = pd.to_datetime(time_frame)
    time_frame = list(time_frame.astype(str))
    time_frame = pd.DataFrame(time_frame, columns=['Time'])
    time_frame = reformed_df(time_frame)
    time_frame = time_frame[~time_frame.Day_of_week.isin(['Saturday','Sunday'])]
    time_frame = time_frame[time_frame.new_time.isin(working_time())] 


    return(time_frame)

sptf = supposed_time_frame(df)
# sptf

def powersource_function(row):
    """
    0 = grid
    1 = gen
    """
    if (row['Ps1_kW']>row['Ps2_kW']):
        return 0
    if (row['Ps2_kW']>row['Ps1_kW']):
        return 1
    return 99999

df['powersource']=df.apply(lambda row:powersource_function(row), axis=1 )

# time frames not logged 
def not_accounted(df1,df2):
    not_accounted = [item for item in df1.index if item not in df2.index]
#     not_accounted = set(df1.index) - set(df2.index)
# not_accounted = not_accounted(sptf,df)

    not_accounted= pd.to_datetime(not_accounted)

    not_accounted = pd.DataFrame(not_accounted,columns=['Time'])

    # not_accounted

    not_accounted = reformed_df(not_accounted)
    return(not_accounted)

not_accounted = not_accounted(sptf,df)

def determinant(row):
    """
    0 = grid
    1 = gen
    """
    
#     full_df_copy['Forward_Fill'] = full_df_copy['powersource'].ffill()
#     full_df_copy['Backward_Fill'] = full_df_copy['powersource'].bfill()
    
    if (row['Backward_Fill']==0) & (row['Forward_Fill']==0):
        return 0
    if (row['Backward_Fill']==1) & (row['Forward_Fill']==1):
        return 1
    return 0


def create_features(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()
    df['hour'] = df.index.hour
    df['minute'] = df.index.minute
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week
    return df

def time_adjust(df):
    df = df.reset_index()
    df['Time'] = df['Time'].astype(str)
    df['Time']= pd.to_datetime(df['Time'])
    df.sort_values(by='Time', inplace=True)
    return df

 def time_indexing(df):
    df = df.set_index('Time')
    df.index = pd.to_datetime(df.index)
    return df

full_df = pd.concat([df, not_accounted], join="outer",ignore_index=False)
# full_df

wanted_columns = ['Time','Freq_Hz', 'Va_V', 'Vb_V', 'Vc_V', 'Vnavg_V', 'Vab_V',
       'Vbc_V', 'Vca_V', 'Vlavg_V', 'Ia_A', 'Ib_A', 'Ic_A', 'Iavg_A', 'PFa',
       'PFb', 'PFc', 'PF', 'I001_A', 'PF001', 'I002_A', 'PF002', 'I003_A',
       'PF003', 'I004_A', 'PF004', 'I005_A', 'PF005', 'I006_A', 'PF006',
       'I007_A', 'PF007', 'I008_A', 'PF008', 'I009_A', 'PF009', 'PFs1', 'PFs2',
       'PFs3', 'Pa_kW', 'Pb_kW', 'Pc_kW', 'P_kW', 'Qa_kvar', 'Qb_kvar',
       'Qc_kvar', 'Q_kvar', 'Sa_kVA', 'Sb_kVA', 'Sc_kVA', 'S_kVA', 'P001_kW',
       'Q001_kvar', 'S001_kVA', 'P002_kW', 'Q002_kvar', 'S002_kVA', 'P003_kW',
       'Q003_kvar', 'S003_kVA', 'P004_kW', 'Q004_kvar', 'S004_kVA', 'P005_kW',
       'Q005_kvar', 'S005_kVA', 'P006_kW', 'Q006_kvar', 'S006_kVA', 'P007_kW',
       'Q007_kvar', 'S007_kVA', 'P008_kW', 'Q008_kvar', 'S008_kVA', 'P009_kW',
       'Q009_kvar', 'S009_kVA', 'Ps1_kW', 'Qs1_kvar', 'Ss1_kVA', 'Ps2_kW',
       'Qs2_kvar', 'Ss2_kVA', 'Ps3_kW', 'Qs3_kvar', 'Ss3_kVA','powersource']
initial_columns = full_df.columns

# removing all unwanted columns
columns_to_drop = list(set(initial_columns) - set(wanted_columns))
full_df.drop(columns_to_drop, axis=1,inplace=True)
# full_df
# df['Time'] =  pd.to_datetime(df['Time'])

not_accounted_df = full_df[full_df['powersource'].isna()]
# not_accounted_df

# time interpolation
full_df_copy = full_df


col_to_interpolate = ['Freq_Hz', 'Va_V', 'Vb_V', 'Vc_V', 'Vnavg_V', 'Vab_V', 'Vbc_V', 'Vca_V', 'Vlavg_V', 'Ia_A', 'Ib_A', 'Ic_A', 'Iavg_A', 'PFa', 'PFb', 'PFc', 'PF', 'I001_A', 'PF001', 'I002_A',
 'PF002', 'I003_A', 'PF003', 'I004_A', 'PF004', 'I005_A', 'PF005', 'I006_A', 'PF006', 'I007_A', 'PF007', 'I008_A', 'PF008', 'I009_A', 'PF009', 'PFs1', 'PFs2', 'PFs3',
  'Qa_kvar', 'Qb_kvar', 'Qc_kvar', 'Q_kvar', 'Sa_kVA', 'Sb_kVA', 'Sc_kVA', 'S_kVA', 'Q001_kvar', 'S001_kVA',
 'Q002_kvar', 'S002_kVA','Q003_kvar', 'S003_kVA',  'Q004_kvar', 'S004_kVA', 'Q005_kvar', 'S005_kVA', 'Q006_kvar', 'S006_kVA',
 'Q007_kvar', 'S007_kVA', 'Q008_kvar', 'S008_kVA',  'Q009_kvar', 'S009_kVA',  'Qs1_kvar', 'Ss1_kVA',  'Qs2_kvar', 'Ss2_kVA',
  'Qs3_kvar', 'Ss3_kVA', 'powersource']

col_for_ml = ['Pa_kW', 'Pb_kW','Pc_kW',  'P_kW', 'P001_kW', 'P002_kW', 'P003_kW', 'P004_kW', 'P005_kW', 'P006_kW', 'P007_kW','P008_kW', 'P009_kW','Ps1_kW','Ps2_kW','Ps3_kW']

# full_df_copy[col_to_interpolate].columns




# full_df_copy

# full_df_copy.reset_index(inplace=True)
# full_df_copy['Time']= pd.to_datetime(full_df_copy['Time'])

# full_df_copy.sort_values(by='Time', inplace=True)

# # df2 = full_df_copy.sort_values(by='Time', inplace=True)

# full_df_copy['Forward_Fill'] = full_df_copy['powersource'].ffill()
# full_df_copy['Backward_Fill'] = full_df_copy['powersource'].bfill()
# full_df_copy['determinant']=full_df_copy.apply(lambda row:determinant(row), axis=1 )

# for i in list(full_df_copy[col_to_interpolate].columns):
#     full_df_copy[i]= full_df_copy[i].interpolate(option='time')

# # full_df_copy.drop(columns=['Forward_Fill','Backward_Fill'], inplace=True)
# # full_df_copy['Interpolate_time']= full_df_copy['Va_V'].interpolate(option='time')
# # full_df_copy['Flag']= np.where(full_df_copy['Va_V'].isna()==True,'missing','data')

# # full_df_copy['WMA'] = full_df_copy['Va_V'].rolling(window=9).apply(lambda x: x[::-1].cumsum().sum() * 2 / 31941  / (31941  + 1))

# pd.set_option('display.max_rows', 1000)
# # full_df_copy2 = full_df_copy[full_df_copy.new_time.isin(working_time())] 
# # full_df_copy2= full_df_copy[~full_df_copy.Day_of_week.isin(['Saturday','Sunday'])] 

# # full_df_copy[['Time','powersource','Forward_Fill','Backward_Fill','determinant','Va_V','Interpolate_time']]
# full_df_copy

### ML MODEL

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

full_df=time_adjust(full_df)
full_df

# not_accounted_time = not_accounted.drop(columns=not_accounted.columns)
# # not_accounted_time

# df2 = df[['Va_V']]

# train = df2.loc[df.index < '2022-09-20 17:00:00']
# test = df2.loc[df.index >= '2022-09-20 17:00:00']


# df2 = create_features(df2)
# not_accounted_time = create_features(not_accounted_time)


# train = create_features(train)
# test = create_features(test)

# FEATURES = ['dayofyear', 'hour', 'minute','dayofweek', 'quarter', 'month', 'year']
# TARGET = 'Va_V'

# X_train = train[FEATURES]
# y_train = train[TARGET]

# X_not_accounted_time=not_accounted_time[FEATURES]

# X_test = test[FEATURES]
# y_test = test[TARGET]

# reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
#                        n_estimators=1000,
#                        early_stopping_rounds=50,
#                        objective='reg:linear',
#                        max_depth=3,
#                        learning_rate=0.01)
# reg.fit(X_train, y_train,
#         eval_set=[(X_train, y_train), (X_test, y_test)],
#         verbose=1000)

# not_accounted_time['Va_V'] = reg.predict(X_not_accounted_time)
# not_accounted_pred = not_accounted_time[['Va_V']]

# full_df_copy_ml = pd.concat([df, not_accounted_pred], join="outer")

# full_df_copy_ml = time_adjust(full_df_copy_ml)
# full_df_copy_ml

not_accounted_time = not_accounted.drop(columns=not_accounted.columns)
# not_accounted_time
for i in col_for_ml:
    df2 = df[[i]]

    train = df2.loc[df.index < '2022-09-20 17:00:00']
    test = df2.loc[df.index >= '2022-09-20 17:00:00']


    df2 = create_features(df2)
    not_accounted_time = create_features(not_accounted_time)


    train = create_features(train)
    test = create_features(test)

    FEATURES = ['dayofyear', 'hour', 'minute','dayofweek', 'quarter', 'month', 'year']
    TARGET = i

    X_train = train[FEATURES]
    y_train = train[TARGET]

    X_not_accounted_time=not_accounted_time[FEATURES]

    X_test = test[FEATURES]
    y_test = test[TARGET]

    reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                           n_estimators=1000,
                           early_stopping_rounds=50,
                           objective='reg:linear',
                           max_depth=3,
                           learning_rate=0.01)
    reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=1000)

    not_accounted_time[i] = reg.predict(X_not_accounted_time)
    not_accounted_pred = not_accounted_time[[i]]

    full_df_copy_ml = pd.concat([df, not_accounted_pred], join="outer")

    full_df_copy_ml = time_adjust(full_df_copy_ml)
    full_df_copy_ml

not_accounted_time

not_accounted_time.isna().any()

full_df_ml = pd.concat([df, not_accounted_time], join="outer",ignore_index=False)

full_df_ml.drop(columns_to_drop, axis=1,inplace=True)
full_df_ml.drop(columns=['hour','minute','dayofweek','quarter','month','year','dayofyear','dayofmonth','weekofyear'],axis=1,inplace=True)
full_df_ml

### Interpolation

full_df_ml.reset_index(inplace=True)
full_df_ml['Time']= pd.to_datetime(full_df_ml['Time'])

full_df_ml.sort_values(by='Time', inplace=True)

# df2 = full_df_copy.sort_values(by='Time', inplace=True)

full_df_ml['Forward_Fill'] = full_df_ml['powersource'].ffill()
full_df_ml['Backward_Fill'] = full_df_ml['powersource'].bfill()
full_df_ml['determinant']=full_df_ml.apply(lambda row:determinant(row), axis=1 )

for i in list(full_df_ml[col_to_interpolate].columns):
    full_df_ml[i]= full_df_ml[i].interpolate(option='time')

# full_df_copy.drop(columns=['Forward_Fill','Backward_Fill'], inplace=True)
# full_df_copy['Interpolate_time']= full_df_copy['Va_V'].interpolate(option='time')
# full_df_copy['Flag']= np.where(full_df_copy['Va_V'].isna()==True,'missing','data')

# full_df_copy['WMA'] = full_df_copy['Va_V'].rolling(window=9).apply(lambda x: x[::-1].cumsum().sum() * 2 / 31941  / (31941  + 1))

pd.set_option('display.max_rows', 1000)
# full_df_copy2 = full_df_copy[full_df_copy.new_time.isin(working_time())] 
# full_df_copy2= full_df_copy[~full_df_copy.Day_of_week.isin(['Saturday','Sunday'])] 

# full_df_copy[['Time','powersource','Forward_Fill','Backward_Fill','determinant','Va_V','Interpolate_time']]
full_df_ml

### Threating working Hours

full = reformed_df(full_df_ml)
full = full[full.new_time.isin(working_time())]
full = time_adjust(full)
full = full[wanted_columns]
full